# Proyecto Mineria de Datos: Credit Scores

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import os

## Datos

In [3]:
os.chdir('/home/juanpalms/Desktop/Mineria de Datos/Proyecto')

In [4]:
os.getcwd()

'/home/juanpalms/Desktop/Mineria de Datos/Proyecto'

In [5]:
datos_train=pd.read_csv('Data/train.csv')

/tmp/ipykernel_16189/2775519410.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  datos_train=pd.read_csv('Data/train.csv')


In [6]:
datos_test=pd.read_csv('Data/test.csv')

In [8]:
datos_train.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


In [11]:
datos_train.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [12]:
datos_train['Month'].head()

0     January
1    February
2       March
3       April
4         May
Name: Month, dtype: object

### Exploracion inicial de un solo mes de los datos para generar el EDA

In [19]:
#Filtro para meses
filtro_enero=datos_train['Month']=='January'

In [34]:
Jan_tr=datos_train[filtro_enero]

In [35]:
Jan_tr.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,10639.000000,12500.000000,12500.000000,12500.00000,12500.000000,12264.000000,12500.000000,12500.000000
mean,4199.465763,17.275120,21.647680,69.46520,21.051440,24.591650,32.291949,1303.781040
std,3195.283895,118.518214,123.789969,455.95698,14.859994,183.422458,5.084327,8118.261086
min,303.645417,0.000000,1.000000,1.00000,-5.000000,0.000000,20.992914,0.000000
25%,1624.265000,3.000000,4.000000,8.00000,10.000000,2.000000,28.110034,29.128806
50%,3081.902500,6.000000,5.000000,13.00000,18.000000,4.000000,32.297912,66.372879
75%,5973.520833,7.000000,7.000000,20.00000,28.000000,8.000000,36.458660,149.904496
max,15204.633333,1779.000000,1479.000000,5788.00000,67.000000,2592.000000,49.564519,82122.000000


ID                      :Represents a unique identification of an entry
Customer_ID             :Represents a unique identification of a person
Month                   :Represents the month of the year
Name                    :Represents the name of a person
Age                     :Represents the age of the person
SSN                     :Represents the social security number of a person
Occupation              :Represents the occupation of the person
Annual_Income           :Represents the annual income of the person
Monthly_Inhand_Salary   :Represents the monthly base salary of a person
Num_Bank_Accounts       :Represents the number of bank accounts a person holds
Num_Credit_Card         :Represents the number of other credit cards held by a person
Interest_Rate           :Represents the interest rate on credit card
Num_of_Loan             :Represents the number of loans taken from the bank
Type_of_Loan            :Represents the types of loan taken by a person
Delay_from_due_date     :Represents the average number of days delayed from the payment date
Num_of_Delayed_Payment  :Represents the average number of payments delayed by a person
Changed_Credit_Limit    :Represents the percentage change in credit card limit
Num_Credit_Inquiries    :Represents the number of credit card inquiries (An inquiry refers to a request to look at                          your credit file and falls into one of two camps: hard or soft. A credit inquiry occurs                            when you apply for a credit card or loan and permit the issuer or lender to check your                            credit. Some inquiries have no effect on your credit, but others can lower your credit                             score.)
Credit_Mix              :Represents the classification of the mix of credits
Outstanding_Debt        :Represents the remaining debt to be paid (in USD)
Credit_Utilization_Ratio:Represents the utilization ratio of credit card
Credit_History_Age      :Represents the age of credit history of the person
Payment_of_Min_Amount   :Represents whether only the minimum amount was paid by the person
Total_EMI_per_month     :Represents the monthly EMI payments (in USD)
Amount_invested_monthly :Represents the monthly amount invested by the customer (in USD)
Payment_Behaviour       :Represents the payment behavior of the customer (in USD)
Monthly_Balance         :Represents the monthly balance amount of the customer (in USD)
Credit_Score            :Represents the bracket of credit score (Poor, Standard, Good)

Profiling de los datos tomando solo ciertas caracteristicas basicas iniciales

In [36]:
#columnas de descripcion general fast EDA
columnas_FEDA=['Occupation','Monthly_Inhand_Salary','Num_Credit_Card','Interest_Rate',
               'Num_of_Loan','Type_of_Loan','Delay_from_due_date','Num_Credit_Inquiries',
               'Credit_Utilization_Ratio','Payment_of_Min_Amount','Credit_Score']

In [37]:
Jan_tr_pr=Jan_tr[columnas_FEDA]

In [42]:
profile_jan=pp.ProfileReport(Jan_tr_pr)

In [44]:
profile_jan.to_file('JanuaryProfiling.html')

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Data Engineering

#Por hacer:
1) Rellenar missing values de variables evidentes
1) Separar la columna de Type_of_Loan: pq?- el credito al consumo no es igual a otro tipo de creditos, tener dummies por tipo de creditos y relacionarlo con otras variables es importante para entender el comportamiento de los clientes. Posiblemente la gente que tiene creditos al consumo es mas propensa a defaultear por lo tanto tienen peor score crediticio.